## Dashboard Creation

In [35]:
#import dashboard_dataset to final_dataset data frame
import pandas as pd
final_dataset = pd.read_csv('./dashboard_dataset.csv')

In [36]:
#import necessary libraries
import panel as pn
pn.extension()
import hvplot.pandas
import matplotlib.pyplot as plt
import holoviews as hv
import plotly.express as px


In [37]:
print(final_dataset.head(10))
print(final_dataset.shape)
print(final_dataset.info())

         date  date_num  month_name  year_num  shop_id  \
0  2013-01-02         2           1         0       59   
1  2013-01-03         3           1         0       25   
2  2013-01-06         6           1         0       25   
3  2013-01-15        15           1         0       25   
4  2013-01-10        10           1         0       25   
5  2013-01-02         2           1         0       25   
6  2013-01-04         4           1         0       25   
7  2013-01-11        11           1         0       25   
8  2013-01-03         3           1         0       25   
9  2013-01-03         3           1         0       25   

                 shop_name  item_id  \
0   Yaroslavl TC" Altair "    22154   
1     Moscow TEC" Atrium "     2552   
2     Moscow TEC" Atrium "     2554   
3     Moscow TEC" Atrium "     2555   
4     Moscow TEC" Atrium "     2564   
5     Moscow TEC" Atrium "     2565   
6     Moscow TEC" Atrium "     2572   
7     Moscow TEC" Atrium "     2572   
8     Mosc

In [38]:
#decoding year_num column and month_name column
final_dataset['year_num'] = final_dataset['year_num'].replace({0: '2013', 1: '2014', 2: '2015'})
final_dataset['month_name'] = final_dataset['month_name'].replace({1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6:'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11:'November', 12: 'December'})

print(final_dataset.head(10))
print(final_dataset.tail(10))   

         date  date_num month_name year_num  shop_id                shop_name  \
0  2013-01-02         2    January     2013       59   Yaroslavl TC" Altair "   
1  2013-01-03         3    January     2013       25     Moscow TEC" Atrium "   
2  2013-01-06         6    January     2013       25     Moscow TEC" Atrium "   
3  2013-01-15        15    January     2013       25     Moscow TEC" Atrium "   
4  2013-01-10        10    January     2013       25     Moscow TEC" Atrium "   
5  2013-01-02         2    January     2013       25     Moscow TEC" Atrium "   
6  2013-01-04         4    January     2013       25     Moscow TEC" Atrium "   
7  2013-01-11        11    January     2013       25     Moscow TEC" Atrium "   
8  2013-01-03         3    January     2013       25     Moscow TEC" Atrium "   
9  2013-01-03         3    January     2013       25     Moscow TEC" Atrium "   

   item_id                                          item_name  category_id  \
0    22154                    

In [39]:
#make data frame for the dashboard interactive
df = final_dataset.copy()

In [40]:
#sales data overview

In [41]:
#total sales (plain text)
total_sales = df['revenue'].sum()
total_sales = round(total_sales, 2)
total_sales = f'${total_sales}'
print(total_sales)

$3409537438.57


In [42]:
print(df.columns)

Index(['date', 'date_num', 'month_name', 'year_num', 'shop_id', 'shop_name',
       'item_id', 'item_name', 'category_id', 'item_category_name',
       'item_price', 'item_cnt_month', 'revenue', 'price_range', 'log_revenue',
       'scaled_revenue'],
      dtype='object')


In [46]:


# Define functions for each visualization

def total_sales():
    # Plain text showing total sales
    total_sales_text = f"Total Sales: ${df['revenue'].sum():,.2f}"
    return pn.panel(total_sales_text)

def sales_data_per_store():
    # Pie chart showing sales data per store using Plotly
    store_sales = df.groupby('shop_name')['revenue'].sum().reset_index()
    fig = px.pie(store_sales, values='revenue', names='shop_name', title='Sales Data per Store')
    return pn.pane.Plotly(fig)

def monthly_sales_analysis():
    # Line charts for mean, median, and standard deviation of monthly sales
    monthly_stats = df.groupby(['year_num', 'month_name'])['revenue'].agg(['mean', 'median', 'std']).reset_index()
    line_chart_mean = monthly_stats.hvplot.line(x='month_name', y='mean', groupby='year_num', title='Mean Monthly Sales')
    line_chart_median = monthly_stats.hvplot.line(x='month_name', y='median', groupby='year_num', title='Median Monthly Sales')
    line_chart_std = monthly_stats.hvplot.line(x='month_name', y='std', groupby='year_num', title='Standard Deviation of Monthly Sales')
    
    return line_chart_mean + line_chart_median + line_chart_std

def yearly_sales_analysis():
    # Line chart for mean sales across the years
    yearly_mean_sales = df.groupby('year_num')['revenue'].mean().hvplot.line(
        title='Mean Sales Across the Years',
        ylabel='Mean Sales',
    )
    return yearly_mean_sales

def item_category_analysis():
    # Pie chart for item category popularity using Plotly Express
    category_popularity = df['item_category_name'].value_counts().reset_index()
    category_popularity.columns = ['item_category_name', 'count']
    
    fig = px.pie(
        category_popularity,
        names='item_category_name',
        values='count',
        title='Item Category Popularity',
        height=400
    )
    
    # Bar chart for average sales per item category
    avg_sales_per_category = df.groupby('item_category_name')['revenue'].mean().reset_index()
    avg_sales_per_category.columns = ['item_category_name', 'avg_revenue']
    
    bar_chart_category = avg_sales_per_category.hvplot.bar(
        x='item_category_name',
        y='avg_revenue',
        title='Average Sales per Item Category',
        xlabel='Item Category',
        ylabel='Average Sales',
        rot=90,
        height=400
    )

    return pn.Column(fig, bar_chart_category)

def average_sales_metrics():
    # Bar chart for average sales per store, line chart for average store sales year-wise,
    # bar chart for average sales per item, and line chart for average item sales year-wise
    avg_sales_per_store = df.groupby('shop_name')['revenue'].mean().hvplot.bar(
        title='Average Sales per Store',
        xlabel='Store',
        ylabel='Average Sales',
        rot=90,
        height=400
    )
    
    avg_store_sales_yearly = df.groupby(['year_num', 'shop_name'])['revenue'].mean().hvplot.line(
        title='Average Store Sales Year-wise',
        ylabel='Average Sales',
        groupby='shop_name',
        height=400
    )
    
    avg_sales_per_item = df.groupby('item_name')['revenue'].mean().hvplot.bar(
        title='Average Sales per Item',
        xlabel='Item',
        ylabel='Average Sales',
        rot=90,
        height=400
    )
    
    avg_item_sales_yearly = df.groupby(['year_num', 'item_name'])['revenue'].mean().hvplot.line(
        title='Average Item Sales Year-wise',
        ylabel='Average Sales',
        groupby='item_name',
        height=400
    )
    
    return avg_sales_per_store + avg_store_sales_yearly + avg_sales_per_item + avg_item_sales_yearly

def monthly_trends():
    # Line chart showing the relationship between month of year and average sales
    monthly_mean_sales = df.groupby(['year_num', 'month_name'])['revenue'].mean().reset_index()
    monthly_trends_chart = monthly_mean_sales.hvplot.line(
        x='month_name',
        y='revenue',
        groupby='year_num',
        title='Monthly Sales Trends',
        xlabel='Month',
        ylabel='Average Sales',
        rot=90,
        height=400
    )
    return monthly_trends_chart

# Additional Insights
def additional_insights():
    insights = df.describe().reset_index()
    insights = insights.rename(columns={'index': 'Metric'})
    return pn.widgets.DataFrame(insights)

# Create the Panel dashboard
dashboard = pn.Tabs(
    ("Total Sales", total_sales()),
    ("Sales Data per Store", sales_data_per_store()),
    ("Monthly Sales Analysis", monthly_sales_analysis()),
    ("Yearly Sales Analysis", yearly_sales_analysis()),
    ("Item Category Analysis", item_category_analysis()),
    ("Average Sales Metrics", average_sales_metrics()),
    ("Monthly Trends", monthly_trends()),
    ("Additional Insights", additional_insights())
)

# Display the dashboard
dashboard.show()


Launching server at http://localhost:59883
